In [1]:
import numpy as np
import pandas as pd
# Importo OS solo para ver la lista de archivos de data
import os

In [2]:
def eliminar_filas_invalidas(df):

    filas=[]

    for i in df.index:

        try:
            float(df.iloc[i]["Cierre"])
        
        except:
            print(i)
            filas.append(i)

    df.drop(filas, axis=0,inplace=True)
    df.reset_index(drop=True)

In [3]:
def formatear_fecha(fecha):

    dict_meses = {
        "ene":"01",
        "feb":"02",
        "mar":"03",
        "abr":"04",
        "may":"05",
        "jun":"06",
        "jul":"07",
        "ago":"08",
        "sept":"09",
        "oct":"10",
        "nov":"11",
        "dic":"12"
    }

    return fecha.replace(fecha.split(" ")[1], dict_meses[fecha.split(" ")[1]])

In [4]:
def main(archivo, indice):
    df = pd.read_csv(f"./data/{indice}/{archivo}")

    #Renombro  las columnas para que sean más accesibles
    df.rename(columns = {"Abrir": "Apertura", "Máx.": "Max","Mín.":"Min","Cierre*":"Cierre"}, inplace=True)

    #Elimino la columna Cierre ajus.** ya que está duplicada
    df.drop(columns=["Cierre ajus.**"], inplace=True)

    #Elimino la última fila que no trae información
    df.drop(df.index[-1], axis=0,inplace=True)

    df["Indice"] = np.full(df.shape[0], fill_value=indice)

    df[["Apertura", "Max", "Min", "Cierre","Volumen"]] = (df[["Apertura", "Max", "Min", "Cierre","Volumen"]].applymap(lambda x : x.replace(".","").replace(",","")))

    eliminar_filas_invalidas(df)

    df["Volumen"] = df["Volumen"].replace("-","0")

    df[["Apertura", "Max", "Min", "Cierre","Volumen"]] = df[["Apertura", "Max", "Min", "Cierre","Volumen"]].astype(float)
    df[["Apertura", "Max", "Min", "Cierre"]] = df[["Apertura", "Max", "Min", "Cierre"]]/100

    df["Cambio"] = df["Cierre"] - df["Apertura"]
    df["Cambio%"] = round(df["Cambio"]*100 / df["Apertura"],3)

    df["Fecha"] = df["Fecha"].map(lambda x : formatear_fecha(x))
    df["Fecha"] = df["Fecha"].map(lambda x : pd.to_datetime(x, format='%d %m %Y')).astype("datetime64[ns]")

    df["weekday"] = df["Fecha"].map(lambda x : x.strftime('%A'))
    df["month"] = df["Fecha"].map(lambda x : x.strftime('%B'))
    df["year"] = df["Fecha"].map(lambda x : x.year )

    return df

In [10]:
dict_ficheros = {carpeta.split("\\")[-1]:file for carpeta, subcarpeta, file in os.walk("./data") if file!=[]}

for carpeta in list(dict_ficheros.keys()):

    if carpeta =="./data" or carpeta == "Airtable": continue
        
    for archivo in dict_ficheros[carpeta]:
        print(archivo)    
        
        if archivo.find("componentes") != -1:
            continue

        print(carpeta)   
        print(archivo)    
        df = main(archivo,carpeta)
        df.to_csv(f'./data/{carpeta}/{archivo}', index=False) 



./data
Airtable
ATX
BFX
BVSP
CASE30
DJI
FCHI
GDAXI
GSPC
HSI
IBEX
INDC.MC
INDD.MC
INDS.MC
IPSA
IXIC
JKSE
JN0U.JO
KS11
MXX
N100
N225
NZ50
OMXC20
OMXSPI
OSEAX
SSMI
STOXX50E
TA125.TA
TWII


In [6]:
#Elimino las carpetas vacias
carpetas_vacias  = {carpeta.split("\\")[-1]:file for carpeta, subcarpeta, file in os.walk("./data") if file==[] }
print(carpetas_vacias)

for carpeta in carpetas_vacias:

    if carpeta =="./data": continue
    
    os.removedirs(f"./data/{carpeta}")


{}
